In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

'''
найти дескрипторы
и
совместить

'''

In [2]:
noises = ['None','gauss', 's&p', 'poisson', 'speckle']

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[tuple(coords)] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[tuple(coords)] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy
    
    return image

# Matcher BRUTEFORCE

In [7]:
@interact_manual(noise=noises, rotate=(0, 360), scale=(0.5, 1.5, 1e-2),
          minHessian=(1, 50000, 1))
def features_1_detect(noise, rotate, scale,
                   minHessian):
    
    
    
    src = cv.imread('grabCut/bike.jpg')
    #src_gray = cv.medianBlur(src_gray, 5)
    src = cv.cvtColor(src, cv.COLOR_BGR2RGB)
    
    src_gray = cv.cvtColor(src, cv.COLOR_RGB2GRAY)
    
    img = src.copy()
    num_rows, num_cols = img.shape[:2]

    rotation_matrix = cv.getRotationMatrix2D((num_cols/2, num_rows/2), rotate, scale)
    
    
    img = noisy(noise, img)
    img = img.astype('uint8')
    
    
    img = cv.warpAffine(img, rotation_matrix, (num_cols, num_rows))
    
    img_gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    
    #-- Step 1: Detect the keypoints using SURF Detector, compute the descriptors
    detector = cv.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    
    keypoints1, descriptors1 = detector.detectAndCompute(src_gray, None)
    keypoints2, descriptors2 = detector.detectAndCompute(img_gray, None)
    #-- Step 2: Matching descriptor vectors with a brute force matcher
    # Since SURF is a floating-point descriptor NORM_L2 is used
    
    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_BRUTEFORCE)
    matches = matcher.match(descriptors1, descriptors2)
    
    #-- Draw matches
    img_matches = np.empty((max(src.shape[0], img.shape[0]), src.shape[1]+img.shape[1], 3), dtype=np.uint8)
    cv.drawMatches(src, keypoints1, img, keypoints2, matches, img_matches)
    
    
    
    
    
    titles = ['img_matches']#['Original Image', 'Tpl', 'C Image', 'C Tpl']
    images = [img_matches]#[src, tpl, dst_src, dst_tpl]
    #fig = plt.gcf()
    #fig.set_size_inches(20, 20)
    for i in range(len(images)):
        #plt.subplot(2, 3, i+1)
        plt.figure(figsize = (20,10))
        plt.imshow(images[i], aspect='auto')
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()
    return (rotate, scale)
    
    
    
    

interactive(children=(Dropdown(description='noise', options=('None', 'gauss', 's&p', 'poisson', 'speckle'), va…

# Matcher_FLANNBASED

In [9]:
@interact_manual(noise=noises, rotate=(0, 360), scale=(0.5, 1.5, 1e-2),
          minHessian=(1, 50000, 1), ratio_thresh=(0.1, 2.0, 0.1))
def features_2_detect(noise, rotate, scale,
                   minHessian, ratio_thresh):
    
    
    
    src = cv.imread('grabCut/bike.jpg')
    #src_gray = cv.medianBlur(src_gray, 5)
    src = cv.cvtColor(src, cv.COLOR_BGR2RGB)
    
    src_gray = cv.cvtColor(src, cv.COLOR_RGB2GRAY)
    
    img = src.copy()
    num_rows, num_cols = img.shape[:2]

    rotation_matrix = cv.getRotationMatrix2D((num_cols/2, num_rows/2), rotate, scale)
    
    
    img = noisy(noise, img)
    img = img.astype('uint8')
    
    
    img = cv.warpAffine(img, rotation_matrix, (num_cols, num_rows))
    
    img_gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    
    detector = cv.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    
    keypoints1, descriptors1 = detector.detectAndCompute(src_gray, None)
    keypoints2, descriptors2 = detector.detectAndCompute(img_gray, None)
    
    #-- Step 2: Matching descriptor vectors with a FLANN based matcher
    # Since SURF is a floating-point descriptor NORM_L2 is used
    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
    knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)
    #-- Filter matches using the Lowe's ratio test
    
    good_matches = []
    for m,n in knn_matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
    #-- Draw matches
    img_matches = np.empty((max(src.shape[0], img.shape[0]), src.shape[1]+img.shape[1], 3), dtype=np.uint8)
    cv.drawMatches(src, keypoints1, img, keypoints2, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    
    
    
    
    titles = ['img_matches']#['Original Image', 'Tpl', 'C Image', 'C Tpl']
    images = [img_matches]#[src, tpl, dst_src, dst_tpl]
    fig = plt.gcf()
    fig.set_size_inches(20, 20)
    for i in range(len(images)):
        #plt.subplot(2, 3, i+1)
        plt.figure(figsize = (20,10))
        plt.imshow(images[i], aspect='auto')
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()
    return (rotate, scale)
    
    
    
    

interactive(children=(Dropdown(description='noise', options=('None', 'gauss', 's&p', 'poisson', 'speckle'), va…